In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import math
from sklearn.preprocessing import StandardScaler
import random
import sklearn
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:
cocomo=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomo81_dataset.csv",header=None)
columns_cocomo=['rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','actual']
cocomo.set_axis(columns_cocomo,axis='columns',inplace=True)
cocomo.set_axis(range(1,64),axis=0 ,inplace=True)
cocomo.rename_axis("Features", axis=1,inplace=True)
cocomo.rename_axis("Projects", axis=0,inplace=True)

X = cocomo.drop('actual',axis=1).values
# Target values y should have shape (n_samples,)
y = cocomo['actual'].values

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
# Split data into train and test sets

# Create a classifier
estimator = SVR(kernel='linear')

# Create SFS model
sfs = SequentialFeatureSelector(estimator,n_features_to_select='auto', direction='forward', cv=5 , scoring='neg_mean_absolute_error', tol=0.1)

# Fit SFS model on training data
sfs.fit(X, y)
print(sfs.get_feature_names_out())
# Transform the data to selected features
X_selected = sfs.transform(X)
print(X_selected)

#=====================================================================================================================

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Step 3: Build the ANN model.
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

    # Step 4: Compile the model.
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 5: Train the model.
    model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=1)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
    
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")   
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

['x0' 'x3' 'x4' 'x5' 'x15']
[[8.80e-01 1.00e+00 1.06e+00 1.15e+00 1.13e+02]
 [8.80e-01 1.00e+00 1.06e+00 1.00e+00 2.93e+02]
 [1.00e+00 1.00e+00 1.00e+00 8.70e-01 1.32e+02]
 [7.50e-01 1.00e+00 1.00e+00 8.70e-01 6.00e+01]
 [8.80e-01 1.00e+00 1.00e+00 8.70e-01 1.60e+01]
 [7.50e-01 1.00e+00 1.21e+00 1.00e+00 4.00e+00]
 [7.50e-01 1.00e+00 1.00e+00 8.70e-01 6.90e+00]
 [1.15e+00 1.66e+00 1.56e+00 1.30e+00 2.20e+01]
 [1.15e+00 1.30e+00 1.21e+00 1.15e+00 3.00e+01]
 [1.40e+00 1.11e+00 1.56e+00 1.00e+00 2.90e+01]
 [1.40e+00 1.11e+00 1.56e+00 1.00e+00 3.20e+01]
 [1.15e+00 1.11e+00 1.06e+00 1.00e+00 3.70e+01]
 [1.15e+00 1.11e+00 1.06e+00 1.15e+00 2.50e+01]
 [1.15e+00 1.30e+00 1.56e+00 1.15e+00 3.00e+00]
 [1.40e+00 1.30e+00 1.06e+00 1.15e+00 3.90e+00]
 [1.40e+00 1.30e+00 1.56e+00 1.00e+00 6.10e+00]
 [1.40e+00 1.30e+00 1.56e+00 1.00e+00 3.60e+00]
 [1.15e+00 1.30e+00 1.21e+00 1.00e+00 3.20e+02]
 [1.15e+00 1.11e+00 1.21e+00 8.70e-01 1.15e+03]
 [1.40e+00 1.11e+00 1.21e+00 1.15e+00 2.99e+02]
 [1.00e+00 1